In [5]:
import numpy as np
import skimage.transform as trans
import skimage.io as io
from keras.preprocessing.image import ImageDataGenerator
import numpy as np 
import os
import glob

import tensorflow as tf
seed_value= 666
os.environ['PYTHONHASHSEED']=str(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)


# data preporcess

In [6]:
"""load data"""
from cfg import *
#patients in data_folcder is cinsistent with traget_folder
non_name_set = set()
PR_name_set = set()
name_set = set()
total_data_path = [non_T1C_data , non_T1_data , non_Flair_data , non_T2_data , PR_T1C_data , PR_T1_data , PR_Flair_data  , PR_T2_data]
count1 = 0
for folder_name in total_data_path :
    for name in os.listdir(folder_name) :
        name = name.strip('.jpg').strip('.json')
        if name == 'ipynb_checkpoint' :
            continue
        name_set.add(name)
        if count1 <= 3 :
            non_name_set.add(name)
        else :
            PR_name_set.add(name)
    count1 += 1
#print(name_set)


In [7]:
total_data_list = []
total_target_list =[] 
for non_name , pr_name in zip(non_name_set , PR_name_set) :
    non_name1 = non_name + ".jpg"
    non_name2 = non_name + ".png"
    pr_name1 = pr_name + ".jpg"
    pr_name2 = pr_name + ".png"
    total_data_list.append(os.path.join(non_T1C_data , non_name1))
    total_data_list.append(os.path.join(non_T1_data , non_name1))
    total_data_list.append(os.path.join(non_Flair_data , non_name1))
    total_data_list.append(os.path.join(non_T2_data , non_name1))
    total_target_list.append(os.path.join(non_T1C_target , non_name2))
    total_target_list.append(os.path.join(non_T1_target , non_name2))
    total_target_list.append(os.path.join(non_Flair_target , non_name2))
    total_target_list.append(os.path.join(non_T2_target , non_name2))
    
    total_data_list.append(os.path.join(PR_T1C_data , pr_name1))
    total_data_list.append(os.path.join(PR_T1_data , pr_name1))
    total_data_list.append(os.path.join(PR_Flair_data , pr_name1))
    total_data_list.append(os.path.join(PR_T2_data , pr_name1))
    total_target_list.append(os.path.join(PR_T1C_target , pr_name2))
    total_target_list.append(os.path.join(PR_T1_target , pr_name2))
    total_target_list.append(os.path.join(PR_Flair_target , pr_name2))
    total_target_list.append(os.path.join(PR_T2_target , pr_name2))   

In [9]:
train_data_list = total_data_list[0 : int(len(total_data_list)*0.9)] #90%data
train_target_list =  total_target_list[0 : int(len(total_target_list)*0.9) ] #90%target

test_data_list = total_data_list[int(len(total_data_list)*0.9) : len(total_data_list)] #90%data
test_target_list = total_target_list[int(len(total_target_list)*0.9) : len(total_target_list)] #90%target

val_img = train_data_list[0 : int(0.2* len(train_data_list))]
val_lbl = train_target_list[0 : int(0.2* len(train_target_list))]
train_data_list = train_data_list[int(0.2* len(train_data_list)) : ]
train_target_list = train_target_list[int(0.2* len(train_target_list)) : ]
print(len(train_data_list))
print(len(train_target_list))
print(len(test_data_list))
print(len(test_target_list))
print(len(val_img))
print(len(val_img))


75
75
11
11
18
18


In [17]:
"""test"""
import cv2

datagen = ImageDataGenerator(rescale=1.0/255.0)
input1 = cv2.imread(train_data_list[0])
input1 = np.expand_dims(input1 , axis= 0 )
train_iterator = datagen.flow(input1, batch_size=1)
print(train_iterator)

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
#mask[mask > 0.5] = 1
#mask[mask <= 0.5] = 0
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomFlip("vertical"),
        layers.RandomZoom(0.4),
        layers.RandomRotation(0.3),
    ]
)

#(train_mode = False) for (test and val)
def train_gen(x_path_list , y_path_list , batch_size , train_mode = True , shuffle = True):
    data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomFlip("vertical"),
        layers.RandomZoom(0.4),
        layers.RandomRotation(0.3),
    ]
    )
    for x , y in zip(x_path_list , y_path_list) :
        x , y = cv2.imread(x) , cv2.imread(y)
        if x.shape != (512 , 512 , 3) :
            x = cv2.resize(x , (512 , 512))
        if y.shape != (512 , 512 , 3) :
            y = cv2.resize(y , (512 ,512))
        x = layers.Rescaling(1.0 / 255.0)(x)
        y = layers.Rescaling(1.0 / 255.0)(y)
        #do without normalization first
        #x = layers.BatchNormalization()
        if train_mode :
            data_loader = ImageDataGenerator(data_augmentation)
            data_loader.flow()
    
    


# Unet Model

In [4]:
import numpy as np 
import os
import skimage.io as io
import skimage.transform as trans
import numpy as np
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras import backend as keras
import tensorflow as tf
"""
input_shape = (1 ,512, 512, 3)
inputs = tf.random.normal(input_shape)
"""
def unet(pretrained_weights = None,input_size = (512 , 512 , 3)):

    #Dropout(rate = 0.5) -> rate: Float between 0 and 1. Fraction of the input units to drop.
    #->  as to prevent from overfitting
    
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    #pool1 -> (_ , 256 , 256 , 64)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    #pool2 -> (_ , 128 , 128 , 128)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    #pool3 -> (_ , 64 , 64 , 256)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)
    #pool4 -> (_ , 32 , 32 , 512)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)
    #drop5 -> (_ ,  32, 32, 1024)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    #up6 (1, 64, 64, 512)
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)
    #conv6 ->(1, 64, 64, 512)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    #up7 (1, 128, 128, 256)
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)
    #conv7 (1, 128, 128, 256)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    #up8 (1, 256, 256, 128)
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    #up9 (1, 512, 512, 64)
    merge9 = concatenate([conv1,up9], axis = 3)
    #merge9 (1, 512, 512, 128)
    conv9 = Conv2D(64, 3, activation = 'relu', padding =  'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)
    #conv10 (1, 512, 512, 1)

    model = Model(inputs, conv10)

    model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    #model.summary()

    if(pretrained_weights):
    	model.load_weights(pretrained_weights)

    return model


(1, 512, 512, 1)


"\ndef unet(pretrained_weights = None,input_size = (512 , 512 , 3)):\n\n    #Dropout(rate = 0.5) -> rate: Float between 0 and 1. Fraction of the input units to drop.\n    #->  as to prevent from overfitting\n    \n    inputs = Input(input_size)\n    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)\n    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)\n    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)\n    #pool1 -> (_ , 256 , 256 , 64)\n    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)\n    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)\n    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)\n    #pool2 -> (_ , 128 , 128 , 128)\n    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)\n    conv3 = Conv2D(256, 3, activation = 

# Training and outcome